# Limpieza Contextual de Texto
Esta función procesa texto crudo para preservar información crítica para análisis comercial. Elimina elementos como URLs y handles de usuario, mientras preserva fechas, horas, precios, porcentajes, hashtags y emoticones.

In [1]:
import re
import json
import emoji

def clean_business_text(text):
    """
    Procesa texto crudo preservando información crítica para análisis comercial.
    
    Args:
        text (str): Texto a procesar
        
    Returns:
        str: Texto procesado que preserva información comercial relevante
    """
    # Paso 1: Eliminar URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Paso 2: Eliminar handles de usuario (@usuario)
    text = re.sub(r'@\w+', '', text)
    
    # Paso 3: Identificar patrones a preservar
    # Buscamos coincidencias para fechas, horas, precios, porcentajes, etc.
    preserved_patterns = []
    
    # Fechas (DD/MM/YYYY)
    date_matches = re.finditer(r'\b\d{1,2}/\d{1,2}/\d{4}\b', text)
    preserved_patterns.extend([(m.start(), m.end(), m.group(0)) for m in date_matches])
    
    # Horas (HH:MM)
    time_matches = re.finditer(r'\b\d{1,2}:\d{2}\b', text)
    preserved_patterns.extend([(m.start(), m.end(), m.group(0)) for m in time_matches])
    
    # Formatos comerciales (2x, $99.99, 15%)
    format_matches = re.finditer(r'(\b\d+x\b|\$\d+(\.\d+)?|\d+(\.\d+)?%)', text)
    preserved_patterns.extend([(m.start(), m.end(), m.group(0)) for m in format_matches])
    
    # Hashtags (#ejemplo)
    hashtag_matches = re.finditer(r'#\w+', text)
    preserved_patterns.extend([(m.start(), m.end(), m.group(0)) for m in hashtag_matches])
    
    # Paso 4: Crear texto limpio 
    result = []
    last_end = 0
    
    # Ordenar patrones a preservar por posición
    preserved_patterns.sort()
    
    for start, end, pattern in preserved_patterns:
        # Procesar texto entre patrones a preservar
        segment = text[last_end:start]
        # Limpiar segmento (eliminar caracteres no deseados)
        segment = re.sub(r'[^\w\s!?%$/]', ' ', segment)
        result.append(segment)
        # Añadir patrón preservado
        result.append(pattern)
        last_end = end
    
    # Procesar texto después del último patrón
    if last_end < len(text):
        segment = text[last_end:]
        segment = re.sub(r'[^\w\s!?%$/]', ' ', segment)
        result.append(segment)
    
    # Paso 5: Normalizar espacios y preservar emojis
    cleaned_text = ''.join(result)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

# Normalización de Números y Unidades
Esta función normaliza números y unidades en el texto, convirtiendo fechas a formato ISO, procesando monedas y reemplazando números genéricos con etiquetas como `<NUM>`. También convierte unidades como `2x` a `2_unidades`.

In [2]:
import dateparser

def normalize_numbers_and_units(text):
    """
    Normaliza números y unidades en el texto, preservando formatos clave para modelos de precios.
    
    Args:
        text (str): Texto a normalizar
        
    Returns:
        str: Texto con números y unidades normalizados
    """
    # Paso 1: Normalizar fechas a formato ISO
    date_pattern = r'\b(\d{1,2}[/\-\.]\d{1,2}[/\-\.]\d{2,4})\b|\b(\d{1,2}\s+de\s+[a-zA-Z]+\s+de\s+\d{2,4})\b|\b([a-zA-Z]+\s+\d{1,2},?\s+\d{2,4})\b'
    
    def convert_date(match):
        date_str = match.group(0)
        try:
            parsed_date = dateparser.parse(date_str, languages=['es', 'en'])
            if parsed_date:
                return parsed_date.strftime('%Y-%m-%d')
            return date_str
        except:
            return date_str
    
    text = re.sub(date_pattern, convert_date, text)
    
    # Paso 2: Normalizar unidades de venta
    unit_pattern = r'\b(\d+)(x|kg|g|ml|l)\b'
    
    def normalize_unit(match):
        number = match.group(1)
        unit = match.group(2)
        if unit == 'x':
            return f"{number}_unidades"
        else:
            return f"{number}_{unit}"
    
    text = re.sub(unit_pattern, normalize_unit, text)
    
    # Paso 3: Procesar monedas
    # USD: $XXX.XX -> <USD>XXX.XX
    text = re.sub(r'\$\s*(\d+(?:\.\d+)?)', r'<USD>\1', text)
    
    # EUR: XXX.XX€ -> <EUR>XXX.XX
    text = re.sub(r'(\d+(?:\.\d+)?)\s*(?:€|EUR)', r'<EUR>\1', text)
    
    # Formato latinoamericano: XXX.XX$ -> <USD>XXX.XX
    text = re.sub(r'(\d+(?:\.\d+)?)\s*\$', r'<USD>\1', text)
    
    # Paso 4: Reemplazar números genéricos con <NUM>
    # Evitar reemplazar números en formatos ya procesados
    # Dividimos el lookbehind complejo en varios lookbehinds de ancho fijo
    try:
        standalone_number_pattern = r'(?<!\d-)(?<!\d/)(?<!\d\.)(?<!\d_)(?<![A-Z]>)(?<!\d)\b\d+(?:\.\d+)?\b(?![-/.]\d|_\w|%|-\d|-\w)'
        text = re.sub(standalone_number_pattern, '<NUM>', text)
    except re.error:
        # Si falla, usamos un método alternativo con marcado y restauración
        # Marcar números que queremos preservar
        text = re.sub(r'(\d+)-(\d+)', r'PRESERVE_\1-PRESERVE_\2', text)
        text = re.sub(r'(\d+)/(\d+)', r'PRESERVE_\1/PRESERVE_\2', text)
        text = re.sub(r'(\d+)\.(\d+)', r'PRESERVE_\1.PRESERVE_\2', text)
        text = re.sub(r'<(USD|EUR)>(\d+(?:\.\d+)?)', r'<\1>PRESERVE_\2', text)
        text = re.sub(r'(\d+)_(\w+)', r'PRESERVE_\1_\2', text)
        text = re.sub(r'(\d+)%', r'PRESERVE_\1%', text)
        
        # Reemplazar números no marcados
        text = re.sub(r'\b\d+\b', '<NUM>', text)
        
        # Restaurar números preservados
        text = re.sub(r'PRESERVE_(\d+)', r'\1', text)
    
    return text

# Normalización de Mayúsculas con Reconocimiento de Entidades
Esta función convierte el texto a minúsculas mientras preserva la capitalización de marcas, hashtags y entidades geopolíticas. Utiliza spaCy para identificar entidades y un diccionario personalizado para marcas no reconocidas.

In [3]:
import spacy
import pandas as pd
import sys
import subprocess

def normalize_case_with_entities(text):
    """
    Normaliza mayúsculas preservando entidades como marcas, hashtags y ubicaciones.
    
    Args:
        text (str): Texto a normalizar
        
    Returns:
        str: Texto con mayúsculas normalizadas
    """
    # Cargar modelo de spaCy para español
    try:
        nlp = spacy.load("es_core_news_lg")
    except:
        # Si el modelo no está disponible, instalarlo
        subprocess.check_call([sys.executable, "-m", "spacy", "download", "es_core_news_lg"])
        nlp = spacy.load("es_core_news_lg")
    
    # Cargar diccionario personalizado de marcas (asumiendo que existe)
    try:
        marcas_df = pd.read_csv('marcas_comerciales.csv')
        custom_brands = set(marcas_df['marca'].str.lower().tolist())
    except:
        # Si no existe el archivo, crear un conjunto vacío
        custom_brands = set()
    
    # Procesar el texto con spaCy
    doc = nlp(text)
    
    # Crear una lista para almacenar las palabras procesadas
    processed_words = []
    entities_spans = []
    
    # Identificar entidades y sus posiciones
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'LOC', 'PRODUCT', 'GPE', 'PERSON']:
            entities_spans.append((ent.start_char, ent.end_char, ent.text))
    
    # Identificar hashtags y preservarlos
    for match in re.finditer(r'#\w+', text):
        entities_spans.append((match.start(), match.end(), match.group()))
    
    # Convertir todo a minúsculas primero
    text_lower = text.lower()
    
    # Reemplazar entidades preservando su capitalización original
    result = []
    last_end = 0
    
    # Ordenar entidades por posición
    entities_spans.sort()
    
    for start, end, entity_text in entities_spans:
        # Añadir texto en minúsculas hasta la entidad
        result.append(text_lower[last_end:start])
        # Añadir la entidad con capitalización original
        result.append(entity_text)
        last_end = end
    
    # Añadir el resto del texto en minúsculas
    if last_end < len(text):
        result.append(text_lower[last_end:])
    
    normalized_text = ''.join(result)
    
    # Verificar palabras contra diccionario personalizado
    words = re.findall(r'\b\w+\b', normalized_text)
    for word in words:
        if word.lower() in custom_brands and word.lower() != word:
            # Restaurar capitalización original de marca
            original_case = text[text.lower().find(word.lower()):text.lower().find(word.lower()) + len(word)]
            normalized_text = normalized_text.replace(word, original_case)
    
    return normalized_text

# Aplicación de Métodos de Procesamiento
Esta sección carga muestras de validación y aplica las funciones de limpieza, normalización de números y unidades, y normalización de mayúsculas. Los resultados se imprimen para cada muestra.

In [4]:
import json

# Load validation samples
with open('validation_samples.json', 'r', encoding='utf-8') as file:
    samples = json.load(file)

# Select a subset of samples to process
sample_phrases = samples[:3] if len(samples) >= 3 else samples

# Function to apply all methods and display results
def display_all_processings(text):
    print(f"Original: {text}")
    print("-" * 80)
    
    if not isinstance(text, str):
        print("ERROR: El texto no es una cadena. Tipo recibido:", type(text))
        return
    
    try:
        cleaned = clean_business_text(text)
        print(f"After clean_business_text:\n{cleaned}")
    except Exception as e:
        print(f"Error in clean_business_text: {e}")
    print("-" * 80)
    
    try:
        normalized_numbers = normalize_numbers_and_units(text)
        print(f"After normalize_numbers_and_units:\n{normalized_numbers}")
    except Exception as e:
        print(f"Error in normalize_numbers_and_units: {e}")
    print("-" * 80)
    
    try:
        normalized_case = normalize_case_with_entities(text)
        print(f"After normalize_case_with_entities:\n{normalized_case}")
    except Exception as e:
        print(f"Error in normalize_case_with_entities: {e}")
    print("=" * 80)

# Process each sample phrase
for i, phrase in enumerate(sample_phrases):
    print(f"\nSample {i+1}:")
    display_all_processings(phrase['input'])



Sample 1:
Original: 🔥¡OFERTA! Compre 2x zapatos Nike a $99.99 (antes $150) 👟. ¡Válido hasta el 30/11/2023! Visita https://marketmind.com/oferta-nike. Atención @MariaP: ¿Envío gratis? 😃 #ModaDeportiva2023.
--------------------------------------------------------------------------------
After clean_business_text:
OFERTA! Compre 2x zapatos Nike a $99.99 antes $150 Válido hasta el 30/11/2023! Visita Atención Envío gratis? #ModaDeportiva2023
--------------------------------------------------------------------------------
After normalize_numbers_and_units:
🔥¡OFERTA! Compre 2_unidades zapatos Nike a <USD>99.99 (antes <USD>150) 👟. ¡Válido hasta el 2023-11-30! Visita https://marketmind.com/oferta-nike. Atención @MariaP: ¿Envío gratis? 😃 #ModaDeportiva2023.
--------------------------------------------------------------------------------
After normalize_case_with_entities:
🔥¡oferta! compre 2x zapatos Nike a $99.99 (antes $150) 👟. ¡válido hasta el 30/11/2023! visita https://marketmind.com/oferta-